In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
from scipy.ndimage.filters import median_filter
%matplotlib inline

In [2]:
X_train_rotate = np.load("/home/ubuntu/X_train_rotate.npy")
target_train = np.load("/home/ubuntu/target_train.npy")

In [3]:
import keras
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation, BatchNormalization
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [4]:
def get_callbacks(filepath, patience=5):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]

file_path = "model_weights.hdf5"

In [6]:
from sklearn.model_selection import StratifiedKFold

import os

for lr in [1e-3,1e-4,1e-5]:
    print ("ir=",lr)
    for drop in [0.2, 0.4, 0.6, 0.8]:
        print ("drop=",drop)
        for batch_size in [72, 64, 48, 24]:
            print("batch_size=",batch_size)
            print('validation results:+++++++++++++++++++++++++++++')
            cvscores = []
            cvscores_train = []
            kfold = StratifiedKFold(n_splits=5, shuffle=True)
            for train_index, test_index in kfold.split(X_train_rotate, target_train):
                gmodel=Sequential()
                gmodel.add(Conv2D(64, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 3)))
                gmodel.add(BatchNormalization())
                gmodel.add(Conv2D(64, kernel_size=(3, 3),activation='relu'))
                gmodel.add(BatchNormalization())
                gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
                gmodel.add(Dropout(drop))
    
                gmodel.add(Conv2D(64, kernel_size=(3, 3),activation='relu'))
                gmodel.add(BatchNormalization())
                gmodel.add(Conv2D(64, kernel_size=(3, 3),activation='relu'))
                gmodel.add(BatchNormalization())
                gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
                gmodel.add(Dropout(drop))
    
                gmodel.add(Flatten())
    
                gmodel.add(Dense(512))
                gmodel.add(Activation('relu'))
                gmodel.add(Dropout(0.2))
    
                gmodel.add(Dense(256))
                gmodel.add(Activation('relu'))
                gmodel.add(Dropout(0.2))
    
                gmodel.add(Dense(1))
                gmodel.add(Activation('sigmoid'))

                gmodel.compile(loss='binary_crossentropy', 
                               optimizer=Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
                               metrics=['accuracy'])
    
                callbacks = get_callbacks(filepath=file_path, patience=5)
                gmodel.fit(X_train_rotate[train_index], target_train[train_index], 
                           batch_size = batch_size,
                           epochs = 10,
                           verbose = 0,
                           validation_data=(X_train_rotate[test_index], target_train[test_index]),
                           callbacks=callbacks)
                gmodel.load_weights(filepath=file_path)
                scores = gmodel.evaluate(X_train_rotate[test_index], target_train[test_index], verbose=0)
                scores_train = gmodel.evaluate(X_train_rotate[train_index], target_train[train_index], verbose=0)
                print ("lr=",lr)
                print ("drop=",drop)
                print("batch_size=",batch_size)
                print('training:', gmodel.metrics_names[0], scores_train[0], gmodel.metrics_names[1], scores_train[1])
                print('testing:', gmodel.metrics_names[0], scores[0], gmodel.metrics_names[1], scores[1])
                cvscores.append(scores[0])
                cvscores_train.append(scores_train[0])
            print("lr=",lr,"drop=",drop,"batch_size=",batch_size,"loss==============","%.2f (+/- %.2f)" % (np.mean(cvscores), np.std(cvscores))) 
            

ir= 0.001
drop= 0.2
batch_size= 72
validation results:+++++++++++++++++++++++++++++
lr= 0.001
drop= 0.2
batch_size= 72
training: loss 0.229035266808 acc 0.902052481164
testing: loss 0.289307525192 acc 0.883696780893
lr= 0.001
drop= 0.2
batch_size= 72
training: loss 0.227391718452 acc 0.900753442453
testing: loss 0.397319652767 acc 0.855659397715
lr= 0.001
drop= 0.2
batch_size= 72
training: loss 0.210154418497 acc 0.910626136659
testing: loss 0.281237755936 acc 0.863966770509
lr= 0.001
drop= 0.2
batch_size= 72
training: loss 0.236198890124 acc 0.90779220773
testing: loss 0.292560074997 acc 0.873180873181
lr= 0.001
drop= 0.2
batch_size= 72
training: loss 7.56725966888 acc 0.53051155544
testing: loss 7.56426754266 acc 0.530697190427
lr= 0.001 drop= 0.2 batch_size= 72 loss============== 1.76 (+/- 2.90)
batch_size= 64
validation results:+++++++++++++++++++++++++++++
lr= 0.001
drop= 0.2
batch_size= 64
training: loss 0.18744264066 acc 0.919199792154
testing: loss 0.287227175691 acc 0.87746625

lr= 0.001
drop= 0.6
batch_size= 48
training: loss 0.28672034685 acc 0.879449207617
testing: loss 0.298493447241 acc 0.882658359294
lr= 0.001
drop= 0.6
batch_size= 48
training: loss 7.52065056354 acc 0.530649350649
testing: loss 7.52237979014 acc 0.530145530146
lr= 0.001
drop= 0.6
batch_size= 48
training: loss 0.23577391619 acc 0.892755128538
testing: loss 0.280542304718 acc 0.87825182102
lr= 0.001 drop= 0.6 batch_size= 48 loss============== 3.20 (+/- 3.55)
batch_size= 24
validation results:+++++++++++++++++++++++++++++
lr= 0.001
drop= 0.6
batch_size= 24
training: loss 0.285920444379 acc 0.871914783076
testing: loss 0.306354173247 acc 0.857736240914
lr= 0.001
drop= 0.6
batch_size= 24
training: loss 0.271834908154 acc 0.871135359834
testing: loss 0.300363004888 acc 0.860851505711
lr= 0.001
drop= 0.6
batch_size= 24
training: loss 8.45787255725 acc 0.469472590283
testing: loss 8.45956298198 acc 0.469366562825
lr= 0.001
drop= 0.6
batch_size= 24
training: loss 0.257968411686 acc 0.8820779220

ResourceExhaustedError: OOM when allocating tensor with shape[72,64,71,71]
	 [[Node: training_84/Adam/gradients/max_pooling2d_169/MaxPool_grad/MaxPoolGrad = MaxPoolGrad[T=DT_FLOAT, _class=["loc:@max_pooling2d_169/MaxPool"], data_format="NHWC", ksize=[1, 2, 2, 1], padding="VALID", strides=[1, 2, 2, 1], _device="/job:localhost/replica:0/task:0/device:GPU:0"](batch_normalization_338/cond/Merge, max_pooling2d_169/MaxPool, training_84/Adam/gradients/AddN_15)]]

Caused by op 'training_84/Adam/gradients/max_pooling2d_169/MaxPool_grad/MaxPoolGrad', defined at:
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/ubuntu/anacon

In [ ]:
''''''X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(X_train_rotate, target_train, random_state=1, train_size=0.9)
#Without denoising, core features.

import os
gmodel=getModel()
history = gmodel.fit(X_train_cv, y_train_cv,
          batch_size=24,
          epochs=20,
          verbose=0,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks_list)

In [ ]:
#gmodel.load_weights(filepath=filepath)
score = gmodel.evaluate(X_train_rotate, target_train, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
#import testing data
X_test = np.load("/home/ubuntu/X_test.npy")
test_id = np.load('/home/ubuntu/test_id.npy')
predicted_test=gmodel.predict_proba(X_test)

In [ ]:
import pandas as pd
submission = pd.DataFrame()
submission['id']=test_id
submission['is_iceberg']=predicted_test.reshape((predicted_test.shape[0]))
submission.to_csv('sub.csv', index=False)

This jupyter notebook is built with these previous contributors:

https://www.kaggle.com/devm2024/keras-model-for-beginners-0-210-on-lb-eda-r-d Keras Model for Beginners (0.210 on LB)+EDA+R&D by DeveshMaheshwari
https://www.kaggle.com/toregil/welcome-to-deep-learning-cnn-99 Welcome to deep learning (CNN 99%) by Peter Grenholm
https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/ Display Deep Learning Model Training History in Keras by Jason Brownlee
https://machinelearningmastery.com/evaluate-performance-deep-learning-models-keras/ Evaluate the Performance Of Deep Learning Models in Keras by Jason Brownlee
https://machinelearningmastery.com/check-point-deep-learning-models-keras/ Add callbacks for best evaluation